###Подготовка библиотек

Установка torchvision

In [0]:
!pip install -U torch torchvision

Импортируем нужные библиотеки, делаем настройки для работы с GPU

In [0]:


from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'



Импортируем нужные библиотеки, проверяем доступность GPU

In [0]:
# we will verify that GPU is enabled for this notebook


import torch
import numpy as np

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

Удалим старую версию Pillow и установим новую Pillow 5.3.0 (в конце необходимо перегрузить runtime)

In [0]:

!pip uninstall -y Pillow
!pip install Pillow==5.3.0
import PIL
print(PIL.PILLOW_VERSION)


Подключаем google drive (для google colab)

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

Расзиповываем архив

In [0]:
!unzip -q /content/gdrive/My\ Drive/simpsons/data/simpsons4.zip


Просматриваем папки из архива


In [0]:
!ls 'train/simpsons_dataset'

Импортируем torch и разрешаем ему доступ к GPU

In [0]:
!nvidia-smi
import torch
torch.cuda.is_available()

Импортируем нужные библиотеки

In [0]:
import pickle
from skimage import io

from tqdm import tqdm, tqdm_notebook
from PIL import Image
from pathlib import Path

from torchvision import transforms
from multiprocessing.pool import ThreadPool
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
from matplotlib import colors, pyplot as plt
%matplotlib inline



# в sklearn не все гладко, чтобы в colab удобно выводить картинки 
# мы будем игнорировать warnings
import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)


In [0]:
# разные режимы датасета 
DATA_MODES = ['train', 'val', 'test']
# все изображения будут масштабированы к размеру RESCALE_SIZExRESCALE_SIZE px
RESCALE_SIZE = 299
# работаем на видеокарте
DEVICE = torch.device("cuda")

Фиксируем генераторы случайных чисел

In [0]:

np.random.seed(0)

torch.manual_seed(0)

torch.cuda.manual_seed(0)

torch.backends.cudnn.deterministic = True

###Подготовка датасета


Загружаем картинки. Переопределяем метод __getitem_ для удобства работы с данной структурой данных. Конвертируем,  немного аугументируем, масштабируем.
 Используем LabelEncoder для преобразования строковых меток классов в id и обратно. Приводим картинки к одному размеру (_prepare_sample).


In [0]:
class SimpsonsDataset(Dataset):
  
    """
    Датасет с картинками, который паралельно подгружает их из папок
    производит скалирование и превращение в торчевые тензоры
    """
    def __init__(self, files, mode):
        super().__init__()
        # список файлов для загрузки
        self.files = sorted(files)
        # режим работы
        self.mode = mode

        if self.mode not in DATA_MODES:
            print(f"{self.mode} is not correct; correct modes: {DATA_MODES}")
            raise NameError

        self.len_ = len(self.files)
     
        self.label_encoder = LabelEncoder()

        if self.mode != 'test':
            self.labels = [path.parent.name for path in self.files]
            self.label_encoder.fit(self.labels)

            with open('label_encoder.pkl', 'wb') as le_dump_file:
                  pickle.dump(self.label_encoder, le_dump_file)
                      
    def __len__(self):
        return self.len_
      
    def load_sample(self, file):
        image = Image.open(file)
        image.load()
        return image
  
    def __getitem__(self, index):
        # для преобразования изображений в тензоры PyTorch и нормализации входа
        
        if self.mode == 'test':
          transform = transforms.Compose([
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) ])
                 
                       
          x = self.load_sample(self.files[index])
          x = self._prepare_sample(x)
          x = np.array(x / 255, dtype='float32')
          x = transform(x)
          return x
        else:
          transform = transforms.Compose([
                                          transforms.ToTensor(),
                                          transforms.ToPILImage(),
                                          transforms.RandomChoice([transforms.ColorJitter(brightness=np.random.uniform(0,0.2),contrast=np.random.uniform(0,0.2),hue=0,saturation=0),
                                                                   transforms.ColorJitter(brightness=0,contrast=0,hue=np.random.uniform(0,0.1),saturation=np.random.uniform(0,0.1)),
                                                                   transforms.RandomHorizontalFlip(),
                                                                   transforms.RandomRotation(np.random.uniform(0,5))]),
                                          
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                          ])
          x = self.load_sample(self.files[index])
          x = self._prepare_sample(x)
          x = np.array(x / 255, dtype='float32')
          x = transform(x)
       
          label = self.labels[index]
          label_id = self.label_encoder.transform([label])
          y = label_id.item()
          return x, y
        
    def _prepare_sample(self, image):
        image = image.resize((RESCALE_SIZE, RESCALE_SIZE))
        return np.array(image)

Показ картинок

In [0]:
def imshow(inp, title=None, plt_ax=plt, default=False):
    """Imshow для тензоров"""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt_ax.imshow(inp)
    if title is not None:
        plt_ax.set_title(title)
    plt_ax.grid(False)

Прописываем пути для файлов

In [0]:
TRAIN_DIR = Path('train/simpsons_dataset')
TEST_DIR = Path('testset/testset')

train_val_files = sorted(list(TRAIN_DIR.rglob('*.jpg')))
test_files = sorted(list(TEST_DIR.rglob('*.jpg')))

Делим исходную  train часть датасета на train и валидационную часть 

In [0]:
from sklearn.model_selection import train_test_split

train_val_labels = [path.parent.name for path in train_val_files]
train_files, val_files = train_test_split(train_val_files, test_size=0.25, \
                                          stratify=train_val_labels, random_state=0)


In [0]:
val_dataset = SimpsonsDataset(val_files, mode='val')

Посмотрим на героев внутри датасета

In [0]:
fig, ax = plt.subplots(nrows=3, ncols=3,figsize=(8, 8), \
                        sharey=True, sharex=True)
for fig_x in ax.flatten():
    random_characters = int(np.random.uniform(0,1000))
    im_val, label = val_dataset[random_characters]
    img_label = " ".join(map(lambda x: x.capitalize(),\
                val_dataset.label_encoder.inverse_transform([label])[0].split('_')))
    imshow(im_val.data.cpu(), \
          title=img_label,plt_ax=fig_x)

### Построение нейросетей




Импортируем предобученную модель Inseption V3 и посмотрим ее архитектуру

In [0]:
from torchvision.models.inception import inception_v3
inception_model0=inception_v3(pretrained=True, aux_logits=False)

print(inception_model0)




Список слоев Inception:

In [0]:
print ('Слои Inception:')
for name, child in inception_model0.named_children():
    for name2, params in child.named_parameters():
        print(name, name2)

Импортируем предобученную модель Resnet50 и посмотрим ее архитектуру

In [0]:
from torchvision.models.resnet import resnet50
resnet_model0=resnet50(pretrained=True)

print(resnet_model0)



Список слоев Resnet:

In [0]:
print ('Слои Resnet:')
for name, child in resnet_model0.named_children():
    for name2, params in child.named_parameters():
        print(name, name2)

Описываем метод fit

In [0]:
def fit_epoch(model, train_loader, criterion, optimizer):
    running_loss = 0.0
    running_corrects = 0
    processed_data = 0
    
  
    for inputs, labels in train_loader:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        preds = torch.argmax(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        processed_data += inputs.size(0)
              
    train_loss = running_loss / processed_data
    train_acc = running_corrects.cpu().numpy() / processed_data
    
    
    return train_loss, train_acc

Описываем метод eval

In [0]:
def eval_epoch(model, val_loader, criterion):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    processed_size = 0

    for inputs, labels in val_loader:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)

        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            preds = torch.argmax(outputs, 1)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        processed_size += inputs.size(0)
    val_loss = running_loss / processed_size
    val_acc = running_corrects.double() / processed_size
    return val_loss, val_acc

Тренируем сеть. Делаем градиентный спуск, оптимизацию. Ставим Scheduler для изменения learning rate каждые 5 эпох. Применяем кроссэнтропию. Считаем loss и accuracy. Сохраняем веса в файл.

In [0]:
def train(train_files, val_files, model, epochs, batch_size, ml):
    
    

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    history = []
    
    
    log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} \
    val_loss {v_loss:0.4f} train_acc {t_acc:0.4f} val_acc {v_acc:0.4f}"

    with tqdm(desc="epoch", total=epochs) as pbar_outer:
      params_to_update = []
      for param in model.parameters():
        if param.requires_grad == True:
          params_to_update.append(param)
      opt = torch.optim.Adam(params_to_update, lr=0.0001)
      scheduler=torch.optim.lr_scheduler.StepLR(opt,7,gamma=0.1)
      criterion = nn.CrossEntropyLoss()
      best_acc=0
      for epoch in range(epochs):
        train_loss, train_acc = fit_epoch(model, train_loader, criterion, opt)
        print("loss", train_loss)
        scheduler.step(train_loss)
        val_loss, val_acc = eval_epoch(model, val_loader, criterion)
        history.append((train_loss, train_acc, val_loss, val_acc))
        
        if train_acc > best_acc:

        # copy the model weights. Cохраним веса в файлы        
          if ml == 'i0':
            torch.save(model.state_dict(), "/content/gdrive/My Drive/simpsons/1/inception_model0_weights.pth") 
          if ml == 'i1':
            torch.save(model.state_dict(), "/content/gdrive/My Drive/simpsons/1/inception_model1_weights.pth") 
          if ml == 'i2':
            torch.save(model.state_dict(), "/content/gdrive/My Drive/simpsons/1/inception_model2_weights.pth") 
          if ml == 'i3':
            torch.save(model.state_dict(), "/content/gdrive/My Drive/simpsons/1/inception_model3_weights.pth") 
          if ml == 'i4':
            torch.save(model.state_dict(), "/content/gdrive/My Drive/simpsons/1/inception_model4_weights.pth") 
          if ml == 'r0':
            torch.save(model.state_dict(), "/content/gdrive/My Drive/simpsons/1/resnet_model0_weights.pth") 
          if ml == 'r1':
            torch.save(model.state_dict(), "/content/gdrive/My Drive/simpsons/1/resnet_model1_weights.pth")
          if ml == 'r2':
            torch.save(model.state_dict(), "/content/gdrive/My Drive/simpsons/1/resnet_model2_weights.pth")
          if ml == 'a':
            torch.save(model.state_dict(), "/content/gdrive/My Drive/simpsons/1/ansamble_weights.pth")   
          best_acc =   train_acc
            



        pbar_outer.update(1)
        tqdm.write(log_template.format(ep=epoch+1, t_loss=train_loss,\
                                       v_loss=val_loss, t_acc=train_acc, v_acc=val_acc))
            
    return history

Делаем предсказание. Применяем softmax

In [0]:
def predict(model, test_loader):
    with torch.no_grad():
        logits = []
    
        for inputs in test_loader:
            inputs = inputs.to(DEVICE)
            model.eval()
            outputs = model(inputs).cpu()
            logits.append(outputs)
            
    probs = nn.functional.softmax(torch.cat(logits), dim=-1).numpy()
    return probs

Посчитаем количество классов в классификаторе

In [0]:
n_classes = len(np.unique(train_val_labels))
print("we will classify :{}".format(n_classes))

Заморозим первые 7 слоев в Inception. Модифицируем слой fc. Установим использование GPU. Посмотрим на модель еще раз.

In [0]:




ct=0
for name, child in inception_model0.named_children():
    ct += 1
    if ct < 7:
        for name2, par in child.named_parameters():
          par.requires_grad = False

inception_model0.fc = nn.Sequential(
    nn.Dropout2d(),
    nn.Linear(2048, out_features=n_classes))

inception_model0=inception_model0.to(DEVICE)

print(inception_model0)

Заморозим первые 7 слоев в Resnet. Разморозим Bottleneck-и: layer1, layer2, layer3. Модифицируем слой fc. Установим использование GPU. Посмотрим на модель еще раз.

In [0]:

ct=0
for name, child in resnet_model0.named_children():
    ct += 1
    if ct < 7:
        for name2, par in child.named_parameters():
          par.requires_grad = False



for param in resnet_model0.fc.parameters() or resnet_model0.layer1.parameters() or resnet_model0.layer2.parameters() or resnet_model0.layer3.parameters():
  param.requires_grad = True



resnet_model0.fc = nn.Sequential(
    nn.Dropout2d(),
    nn.Linear(resnet_model0.fc.in_features, out_features=n_classes)
    )

resnet_model0=resnet_model0.to(DEVICE)

print("we will classify :{}".format(n_classes))
print(resnet_model0)

In [0]:
#val_dataset = SimpsonsDataset(val_files, mode='val')
#train_dataset = SimpsonsDataset(train_files, mode='train')

#train(train_dataset, val_dataset, model=inception_model0, epochs=1, batch_size=128, ml='i0')
#train(train_dataset, val_dataset, model=resnet_model0, epochs=1, batch_size=128, ml='r0')

torch.save(resnet_model0.state_dict(), "/content/gdrive/My Drive/simpsons/1/r0_weights.pth")
torch.save(inception_model0.state_dict(), "/content/gdrive/My Drive/simpsons/1/i0_weights.pth")

In [0]:
label_encoder = pickle.load(open("label_encoder.pkl", 'rb'))

###Обучение нейросетей

Запустим обучение ансамбля

In [0]:
from sklearn.model_selection import StratifiedKFold

n_folds = 3
skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=0)

for fold, (train_idx, val_idx) in enumerate(skf.split(train_val_files, label_encoder.transform(train_val_labels))):
    print('\nFOLD', fold+1)
    val_dataset = SimpsonsDataset(np.array(train_val_files)[val_idx], mode='val')
    train_dataset = SimpsonsDataset(np.array(train_val_files)[train_idx], mode='train')
    
    

    print('Inception model ',str(fold+1))
    inception_model0.load_state_dict(torch.load( "/content/gdrive/My Drive/simpsons/1/i0_weights.pth"))
    history=train(train_dataset, val_dataset, model=inception_model0, epochs=15, batch_size=128, ml=('i'+str(fold)))

    plt.figure(figsize=(15, 9))
    plt.title('Обучение Inception ', str(fold+1))
    loss0, acc0, val_loss0, val_acc0 = zip(*history)
    plt.plot(loss0, label="train_loss")
    plt.plot(val_loss0, label="val_loss")

    print('Resnet model ', str(fold+1))
    resnet_model0.load_state_dict(torch.load("/content/gdrive/My Drive/simpsons/1/r0_weights.pth"))
      
    history=train(train_dataset, val_dataset, model=resnet_model0, epochs=15, batch_size=128, ml=('r'+str(fold)))

    plt.figure(figsize=(15, 9))
    plt.title('Обучение Resnet ', str(fold+1))
    loss0, acc0, val_loss0, val_acc0 = zip(*history)
    plt.plot(loss0, label="train_loss")
    plt.plot(val_loss0, label="val_loss")

    
    
    
    

Загрузка весов обученных моделей

In [0]:


# Если надо, загружаем сохраненные состояния весов нейросетей, пропуская блок "Обучение нейросетей". Во избежание утери данных при повторной распаковке архива, файлы с весами
# храним на google drive.

inception_model1 = inception_model0
inception_model2 = inception_model0

inception_model0.load_state_dict(torch.load("/content/gdrive/My Drive/simpsons/1/inception_model0_weights.pth"))
inception_model1.load_state_dict(torch.load("/content/gdrive/My Drive/simpsons/1/inception_model1_weights.pth"))
inception_model2.load_state_dict(torch.load("/content/gdrive/My Drive/simpsons/1/inception_model2_weights.pth"))

resnet_model1 = resnet_model0
resnet_model2 = resnet_model0

resnet_model0.load_state_dict(torch.load("/content/gdrive/My Drive/simpsons/1/resnet_model0_weights.pth"))
resnet_model1.load_state_dict(torch.load("/content/gdrive/My Drive/simpsons/1/resnet_model1_weights.pth"))
resnet_model2.load_state_dict(torch.load("/content/gdrive/My Drive/simpsons/1/resnet_model2_weights.pth"))

#inception_model0.eval()# переключаем нейросеть в режим обучения



###Обучаем новую нейросеть находить лучшие предсказания из ансамбля

In [0]:
class MyEnsemble(nn.Module):   
    def __init__(self, inception_model0, inception_model1, inception_model2, resnet_model0, resnet_model1, resnet_model2, n_classes):
        super(MyEnsemble, self).__init__()
        self.inception_model0 = inception_model0
        self.inception_model1 = inception_model1
        self.inception_model2 = inception_model2

        self.resnet_model0 = resnet_model0
        self.resnet_model1 = resnet_model1
        self.resnet_model2 = resnet_model2

        self.classifier = nn.Linear(n_classes * 6, n_classes)
        
    def forward(self, x):
        x1 = self.inception_model0(x)
        x2 = self.inception_model1(x)
        x3 = self.inception_model2(x)

        x4 = self.resnet_model0(x)
        x5 = self.resnet_model1(x)
        x6 = self.resnet_model2(x)


        x = torch.cat((x1, x2, x3, x4, x5, x6), dim=1)
        x = self.classifier(x)
        return x

In [0]:
model_ensemble = MyEnsemble(inception_model0, inception_model1, inception_model2, resnet_model0, resnet_model1, resnet_model2, n_classes=42).to(DEVICE)

In [0]:
# замораживаем параметры (веса) не входящие в layers_to_unfreeze
for param in model_ensemble.parameters():
    param.requires_grad = False

for param in model_ensemble.classifier.parameters():
    param.requires_grad = True


In [0]:
val_dataset = SimpsonsDataset(val_files, mode='val')
    
train_dataset = SimpsonsDataset(train_files, mode='train')

In [0]:
#Загрузить веса
#model_ensemble.load_state_dict(torch.load("/content/gdrive/My Drive/simpsons/1/ansamble_weights.pth"))

In [0]:
history = train(train_dataset, val_dataset, model=model_ensemble, epochs=15, batch_size=128, ml='a')

In [0]:
loss, acc, val_loss, val_acc = zip(*history)
plt.figure(figsize=(15, 9))
plt.title('Обучение Ansamble')
plt.plot(loss, label="train_loss")
plt.plot(val_loss, label="val_loss")
plt.legend(loc='best')
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()

###Делаем предсказания

Предсказание одной картинки

In [0]:
def predict_one_sample(model, inputs, device=DEVICE):
    """Предсказание, для одной картинки"""
    with torch.no_grad():
        inputs = inputs.to(device)
        model.eval()
        logit = model(inputs).cpu()
        probs = torch.nn.functional.softmax(logit, dim=-1).numpy()
    return probs
random_characters = int(np.random.uniform(0,1000))
ex_img, true_label = val_dataset[random_characters]
probs_im = predict_one_sample(model_ensemble, ex_img.unsqueeze(0))


In [0]:
#random_characters = int(np.random.uniform(0,1000))
#ex_img, true_label = val_dataset[random_characters]
#probs_im = predict_one_sample(model_ensemble, ex_img.unsqueeze(0))

Посмотрим на предсказания на валидационной выборке

In [0]:
idxs = list(map(int, np.random.uniform(0,1000, 20)))
imgs = [val_dataset[id][0].unsqueeze(0) for id in idxs]

probs_ims = predict(model_ensemble, imgs)

  


Вычислим целевую метрику F1-score на валидационной выборке.

In [0]:
y_pred = np.argmax(probs_ims,-1)

actual_labels = [val_dataset[id][1] for id in idxs]

preds_class = [label_encoder.classes_[i] for i in y_pred]

print("Предсказание: ", y_pred)
print ("Правильный ответ: ", actual_labels)

from sklearn.metrics import f1_score
print("f1_score: ", f1_score(actual_labels, y_pred, average='macro'))

Сделаем визуализацию,  чтобы посмотреть насколько сеть уверена в своих ответах.

In [0]:
import matplotlib.patches as patches
from matplotlib.font_manager import FontProperties

fig, ax = plt.subplots(nrows=3, ncols=3,figsize=(12, 12), \
                        sharey=True, sharex=True)
for fig_x in ax.flatten():
    random_characters = int(np.random.uniform(0,1000))
    im_val, label = val_dataset[random_characters]
    img_label = " ".join(map(lambda x: x.capitalize(),\
                val_dataset.label_encoder.inverse_transform([label])[0].split('_')))
    
    

    imshow(im_val.data.cpu(), \
          title=img_label,plt_ax=fig_x)
    
    actual_text = "Actual : {}".format(img_label)
            
    fig_x.add_patch(patches.Rectangle((0, 53),86,35,color='white'))
    font0 = FontProperties()
    font = font0.copy()
    font.set_family("fantasy")
    prob_pred = predict_one_sample(model_ensemble, im_val.unsqueeze(0))
    predicted_proba = np.max(prob_pred)*100
    y_pred = np.argmax(prob_pred)
    
    predicted_label = label_encoder.classes_[y_pred]
    predicted_label = predicted_label[:len(predicted_label)//2] + '\n' + predicted_label[len(predicted_label)//2:]
    predicted_text = "{} : {:.0f}%".format(predicted_label,predicted_proba)
            
    fig_x.text(1, 59, predicted_text , horizontalalignment='left', fontproperties=font,
                    verticalalignment='top',fontsize=8, color='black',fontweight='bold')

Confusion matrix, accurancy for each class

In [0]:
import itertools
from sklearn.metrics import confusion_matrix
    
   
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    cm = cm.T
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    
    plt.figure(figsize=(16,11))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')

    plt.tight_layout()
    
def show_confusion_matrix_fucn(model):
    """Построить и посчитать точность классов по confusion matrix"""
    y_test_all = torch.Tensor().long()
    predictions_all = torch.Tensor().long()

    # Пройдём по всему validation датасету и запишем ответы сети
    with torch.no_grad():
        for inputs, labels in dataloaders['val']:
            predictions = model(inputs.to(DEVICE))
            y_test = labels
            _, predictions = torch.max(predictions.cpu(), 1)

            # Аналог append для list
            y_test_all = torch.cat((y_test_all, y_test), 0)
            predictions_all = torch.cat((predictions_all, predictions), 0)

    feature_names = sorted(set(dataloaders['val'].dataset.labels))

    y_test_all = y_test_all.numpy()
    predictions_all = predictions_all.numpy()

    # Функция из sklearn, создаёт confusion матрицу
    cm = confusion_matrix(y_test_all, predictions_all, np.arange(n_classes))
    # Выведем её
    plot_confusion_matrix(cm, feature_names, normalize=True)
    
    return y_test_all, predictions_all
  
def accurancy_for_each_class(y_test_all, predictions_all):
    class_correct = [0 for i in range(n_classes)]
    class_total = [0 for i in range(n_classes)]
    feature_names = sorted(set(dataloaders['val'].dataset.labels))

    c = (predictions_all == y_test_all).squeeze()
    for i in range(len(predictions_all)):
        label = predictions_all[i]            
        class_correct[label] += c[i].item()
        class_total[label] += 1

    print(class_total)
    print(len(class_total))

    for i in range(n_classes):
        print('Accuracy of %5s : %2d %%' % (
            (feature_names[i], (100 * class_correct[i] / class_total[i]) if class_total[i] != 0 else -1)))
    

In [0]:
BATCH_SIZE = 128
# DataLoader достаёт данные из dataset батчами
dataloaders = {'train': DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True),
               'val': DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)}
dataset_sizes = {'train': len(train_dataset), 'val':len(val_dataset) }
print ('Для сети model_ensemble:')
y_test_all, predictions_all = show_confusion_matrix_fucn(model_ensemble)
# Выведем точность для каждого класса
accurancy_for_each_class(y_test_all, predictions_all)

Submit на Kaggle

In [0]:
test_dataset = SimpsonsDataset(test_files, mode="test")
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=64)
probs = predict(model_ensemble, test_loader)


In [0]:
preds = label_encoder.inverse_transform(np.argmax(probs, axis=1))
test_filenames = [path.name for path in test_dataset.files]

In [0]:
! ls 

In [0]:
import pandas as pd
#my_submit = pd.read_csv("gdrive/My Drive/simpsons/data/labels.csv")
my_submit = pd.DataFrame({'Id': test_filenames, 'Expected': preds})
my_submit.head()

In [0]:
my_submit.to_csv('gdrive/My Drive/simpsons/ansamble_cnn_baseline.csv', index=False)